In [1]:
import pandas as pd
pd.set_option('display.max_rows', 100)

FormDict={'HEXOSE':'C6H10O5','HEXNAc':'C8H13NO5','Neu5Ac':'C11H17NO8'}
line='Na1C59N4H98O44   1589.545162 HEXOSE:4,HEXNAc:3,Neu5Ac:1,H2O'

def line_comprehension(line):
    line.split()[2].split(',')[0].split(':')
    sample_list=[]
    #print(line.split()[2].split(','))
    for f in line.split()[2].split(','):
        try:
            #print(f.split(':')[0],f.split(':')[1])
            #print(int(f.split(':')[1]))
            for q in range(int(f.split(':')[1])):
                sample_list.append(f.split(':')[0])
        except:
            pass
    return sample_list
sample_list=line_comprehension(line)
#print(sample_list)
def substract_formulas(a,b):
    from pyteomics import mass

    '''Function takes two chemical formulas and substracts them elementwise: 'C2H4'-'H4C'='C'. 
    Returns string'''
    result={}
    S=''
    A=mass.Composition(formula=a)
    B=mass.Composition(formula=b)
    keys=set(list(A.keys())+list(B.keys()))
    for key in keys:
        if key in A:
            aa=A[key]
        else:
            aa=0
        if key in B:
            bb=B[key]
        else:
            bb=0
            
        result[key]=aa-bb
        S=S+key+str(result[key])
    return S     
def makefragformdict(sample_list,additive,subtractive=''):
    from itertools import combinations
    list_combinations = []

    for n in range(len(sample_list)+1):
        list_combinations += list(combinations(sample_list, n))
        #print(n,list(combinations(sample_list, n)))
    list_combinations=set(list_combinations)
    formdict={}
    #print(list(combinations_with_replacement(sample_list, 2)))
    from pyteomics import mass

    for f in list_combinations:
        L=additive
        for q in f:
           L=L+FormDict[q]
        L=substract_formulas(L,subtractive)
        monoisotopic=mass.calculate_mass(formula=L)

        formdict[f]=[monoisotopic,L]
        #print('H'.join(map(str, f)))
    return formdict
def FragID(fragdf,foundfragments,fragtype,delta): 
    Dict={}
    for f in foundfragments:
        try:
            frag=(fragdf[(f-delta<fragdf.mass) & (fragdf.mass<f+delta)])
            L=[frag.mass.values[0]]
            d=float(frag.mass.values[0])-float(f)
            L.append(d)
            L.append(frag.composition.values[0])
            L.append(frag.formula.values[0])
            L.append(fragtype)  
            Dict[f]=L
        except:
            pass
    return Dict

In [25]:
ObservedFragments=[314.0841,336.0742,477.1453,679.2168,697.2274,771.2642,841.2697,915.306436,933.31700,949.29094,
                975.32757,991.30150,1077.388,1136.396373,1137.380389,1152.37031,1153.354326,1175.3585,
                1197.401518,1213.375455,1280.438632,1298.4492,1314.423134,1545.554784,1561.528721,1566.554844,
                1589.544613,1605.51855,1611.526557,1627.500495,1643.473884]

delta=0.1
aglycone='H2O'
#aglycone='C2H5OH'
emass=0.000548579909

FragParams={'B':['H',''],'B+Na':['Na',''],'B+K':['K',''],'B-W':['H','H2O'],'B-W+Na':['Na','H2O'],'B-W+K':['K','H2O'],
            'C':['H3O',''],'C+Na':['H3ONa',''],'C+K':['H3OK',''],'C-W':['H3O','H2O'],'C-W+Na':['H3ONa','H2O'],
            'C-W+K':['H3OK','H2O'],'Y':[aglycone+'H',''],'Y+Na':[aglycone+'Na',''],'Y+K':[aglycone+'K',''],'Y-W':[aglycone+'H','H2O'],
            'Y+Na-CO2':[aglycone+'Na','CO2'],'Y+K-CO2':[aglycone+'K','CO2'],
            'Y-W+Na':[aglycone+'Na','H2O'],'Y-W+K':[aglycone+'K','H2O'],'Y+2Na':[aglycone+'Na2','H'],'Y+2K':[aglycone+'K2','H'],'Y+Na+K':['HONaK',''],
            'A24':['H3O','C6H11NO4'],'A24+Na':['H2ONa','C6H11NO4'],'A24+K':['H2OK','C6H11NO4'],
            'A02':['H3O','C4H7NO2'],'A02+Na':['H2ONa','C4H7NO2'],'A02+K':['H2OK','C4H7NO2'],}
DFlist=[]
for f in FragParams.keys():
    try:
        '''formdict=makefragformdict(sample_list,FragParams[f][0],FragParams[f][1])
        DF=pd.DataFrame(formdict.items(),columns = ['composition','mass'])
        df=pd.DataFrame.from_dict(FragID(DF,ObservedFragments,f,delta), orient='index',columns=['mass','delta','composition', 'fragment_type'])
        DFlist.append(df)'''
        
        formdict=makefragformdict(sample_list,FragParams[f][0],FragParams[f][1])
        #print(formdict)
        DF=pd.DataFrame.from_dict(formdict, orient='index',columns=['mass','formula']).reset_index().rename(columns={"index": "composition"})
        #print(DF.head(1))
        #print(FragID(DF,ObservedFragments,f,delta))
        df=pd.DataFrame.from_dict(FragID(DF,ObservedFragments,f,delta), orient='index',columns=['calculated','delta','composition','formula', 'fragment_type']).reset_index().rename(columns={"index": "observed"})
        DFlist.append(df)

    except:
        pass


Res=pd.concat(DFlist, axis=0, ignore_index=False).sort_values(by=['calculated','delta'])
#Res['calculated']=Res['calculated']-emass
Res.eval("delta = delta -"+ str(emass), inplace=True)
Res.eval("calculated = calculated -"+ str(emass), inplace=True)
Res

C:\Users\pkito\anaconda3\lib\site-packages\pyteomics\mass\mass.py:475: FutureWarning: Returning a signed value. The default will change in the future. Specify `absolute` kwarg to suppress this warning
  warnings.warn('Returning a signed value. The default will change in the future.'
C:\Users\pkito\anaconda3\lib\site-packages\pyteomics\mass\mass.py:475: FutureWarning: Returning a signed value. The default will change in the future. Specify `absolute` kwarg to suppress this warning
  warnings.warn('Returning a signed value. The default will change in the future.'
C:\Users\pkito\anaconda3\lib\site-packages\pyteomics\mass\mass.py:475: FutureWarning: Returning a signed value. The default will change in the future. Specify `absolute` kwarg to suppress this warning
  warnings.warn('Returning a signed value. The default will change in the future.'
C:\Users\pkito\anaconda3\lib\site-packages\pyteomics\mass\mass.py:475: FutureWarning: Returning a signed value. The default will change in the futur

observed   calculated         delta  \
0   314.084100   314.084637  5.372075e-04   
0   314.084100   314.084637  5.372075e-04   
0   477.145300   477.145286 -1.434197e-05   
0   477.145300   477.171501  2.620079e-02   
1   679.216800   679.216833  3.314547e-05   
1   679.216800   679.216833  3.314547e-05   
0   697.227400   697.227398 -2.170829e-06   
1   771.264200   771.264177 -2.273929e-05   
2   841.269700   841.269657 -4.343603e-05   
2   841.269700   841.269657 -4.343603e-05   
3   915.306436   915.306436 -4.488888e-09   
3   915.306436   915.306436 -4.488888e-09   
2   933.317000   933.317001  6.792110e-07   
0   949.290940   949.290938 -1.921689e-06   
0   975.327570   975.327565 -4.637089e-06   
0   975.327570   975.351375  2.380481e-02   
0   991.301500   991.301503  2.762011e-06   
0   991.301500   991.325312  2.381221e-02   
4  1077.388000  1077.359259 -2.874059e-02   
4  1077.388000  1077.359259 -2.874059e-02   
3  1136.396373  1136.396373  1.987208e-07   
1  1137.380389  1137.380389 -2.185891e-07   
1  1137.380389  1137.404198  2.380923e-02   
1  1152.370310  1152.370311  5.978209e-07   
1  1153.354326  1153.354326  1.805111e-07   
1  1153.354326  1153.378136  2.380963e-02   
0  1175.358500  1175.419574  6.107390e-02   
0  1197.401518  1197.401518  1.488111e-07   
0  1213.375455  1213.375456  5.479108e-07   
5  1280.438632  1280.438632 -6.647902e-08   
5  1280.438632  1280.438632 -6.647902e-08   
4  1298.449200  1298.449197 -3.382779e-06   
2  1314.423134  1314.423134  1.632115e-08   
0  1545.554784  1545.554784 -1.154292e-07   
0  1561.528721  1561.528721  2.836710e-07   
5  1589.544613  1589.544613  1.236909e-07   
3  1605.518550  1605.518551  5.227908e-07   
0  1611.526557  1611.526557  3.725211e-07   
0  1627.500495  1627.500495 -2.283791e-07   
0  1643.473884  1643.474432  5.481707e-04   

                                         composition           formula  \
0                                          (Neu5Ac,)     N1Na1O8H17C11   
0                                          (Neu5Ac,)     N1Na1O8H17C11   
0                                   (HEXOSE, Neu5Ac)    N1Na1O13H28C17   
0                                   (HEXNAc, Neu5Ac)       C19O12H29N2   
1                           (HEXOSE, HEXNAc, Neu5Ac)    N2Na1O18H40C25   
1                           (HEXOSE, HEXNAc, Neu5Ac)    N2Na1O18H40C25   
0                           (HEXOSE, HEXNAc, Neu5Ac)    N2Na1O19H42C25   
1                   (HEXOSE, HEXOSE, HEXNAc, HEXNAc)    N2Na1O21H48C28   
2                   (HEXOSE, HEXOSE, HEXNAc, Neu5Ac)    N2Na1O23H50C31   
2                   (HEXOSE, HEXOSE, HEXNAc, Neu5Ac)    N2Na1O23H50C31   
3           (HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNAc)    N2Na1O25H56C34   
3           (HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNAc)    N2Na1O25H56C34   
2           (HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNAc)    N2Na1O26H58C34   
0           (HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNAc)     N2K1O26H58C34   
0   (HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNAc, HEXNAc)    N2Na1O27H60C36   
0           (HEXOSE, HEXOSE, HEXNAc, HEXNAc, HEXNAc)    N3Na1O26H62C36   
0   (HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNAc, HEXNAc)     N2K1O27H60C36   
0           (HEXOSE, HEXOSE, HEXNAc, HEXNAc, HEXNAc)     N3K1O26H62C36   
4   (HEXOSE, HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNAc)    N2Na1O30H66C40   
4   (HEXOSE, HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNAc)    N2Na1O30H66C40   
3   (HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNAc, HEXNAc)    N3Na1O31H71C42   
1  (HEXOSE, HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNA...    N2Na1O32H70C42   
1   (HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNAc, HEXNAc)    N3Na1O31H72C42   
1   (HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNAc, HEXNAc)     N3K1O31H71C42   
1  (HEXOSE, HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNA...     N2K1O32H70C42   
1   (HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNAc, HEXNAc)     N3K1O31H72C42   
0  (HEXOSE, HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNA...       C44N2H75O34   
0  (HEXOSE, HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNA...    N2Na1O34H74C44   
0  (HEXOSE, HE

In [26]:
len(set(Res.observed))

29

In [5]:
Res.to_csv('fragmentation.csv')


In [6]:
FragParams.keys()

dict_keys(['B', 'B+Na', 'B+K', 'B-W', 'B-W+Na', 'B-W+K', 'C', 'C+Na', 'C+K', 'C-W', 'C-W+Na', 'C-W+K', 'Y', 'Y+Na', 'Y+K', 'Y-W', 'Y+Na-CO2', 'Y+K-CO2', 'Y-W+Na', 'Y-W+K', 'Y+2Na', 'Y+2K', 'Y+Na+K', 'A24', 'A24+Na', 'A24+K', 'A02', 'A02+Na', 'A02+K'])

In [7]:
f='Y+Na'
DFlist=[]
formdict=makefragformdict(sample_list,FragParams[f][0],FragParams[f][1])
#print(formdict)
DF=pd.DataFrame.from_dict(formdict, orient='index',columns=['mass','formula']).reset_index().rename(columns={"index": "composition"})
#print(DF.head(1))
#print(FragID(DF,ObservedFragments,f,delta))
df=pd.DataFrame.from_dict(FragID(DF,ObservedFragments,f,delta), orient='index',columns=['calculated','delta','composition','formula', 'fragment_type']).reset_index().rename(columns={"index": "observed"})
DFlist.append(df)
DFlist

[      observed   calculated     delta  \
 0   697.227400   697.227946  0.000546   
 1   771.264200   771.264726  0.000526   
 2   933.317000   933.317549  0.000549   
 3  1136.396373  1136.396922  0.000549   
 4  1298.449200  1298.449745  0.000545   
 5  1589.544613  1589.545162  0.000549   
 
                                          composition         formula  \
 0                           (HEXOSE, HEXNAc, Neu5Ac)  N2Na1O19H42C25   
 1                   (HEXOSE, HEXOSE, HEXNAc, HEXNAc)  N2Na1O21H48C28   
 2           (HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNAc)  N2Na1O26H58C34   
 3   (HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNAc, HEXNAc)  N3Na1O31H71C42   
 4  (HEXOSE, HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNA...  N3Na1O36H81C48   
 5  (HEXOSE, HEXOSE, HEXOSE, HEXOSE, HEXNAc, HEXNA...  N4Na1O44H98C59   
 
   fragment_type  
 0          Y+Na  
 1          Y+Na  
 2          Y+Na  
 3          Y+Na  
 4          Y+Na  
 5          Y+Na  ]

In [50]:
This script consists of 10 nested loops so processing time is proportional to the product of the number of iterations for each fragment defined by "max number"-"min number"+1. To avoid long processing times use reasonable ranges for the fragments. 
To limit output select mass limits.

Each field is customizable. You may save the parameters to a file in "DB/settings" folder. To retrieve settings use "load settings" button. 
The results of calculations are saved as csv files in "results" folder

General principles of oligosaccharide construction:
- aglycone is represented by original complete structure;
- bivalent monomers (such as hexose,pentose...) and monovalent substituents (such as acetate, methyl...) are represented by molecular formula LESS WATER.

Common fragments:
HEXOSE   C6H10O5   HEXNAc   C8H13NO5   deoxy-HEX   C6H10O4   Neu5Ac   C11H17NO8   Me   CH2   Ac   C2H2O 
Octanoyl   C8H14O   IsoButuryl   C4H6O   Succinyl   C4H4O3 

Common aglycones:
WATER   H2O   Methanol   C2H6O   Glyceric Acid  C3H6O4

FRAGMENTATION FUNCTION
Currently, fragmentation only works for free oligosaccharides consisting of 3 types of monosaccharides: 
HEXOSE, HEXNAc and Neu5Ac
Paste a comma-separated list of masses of fragments observed in MSMS spectrum in bottom panel, 
paste a line containig the composition of oligosaccharide in top panel in a following format:
Na1C59N4H98O44   1589.545162 HEXOSE:4,HEXNAc:3,Neu5Ac:1,H2O


Help on method reset_index in module pandas.core.frame:

reset_index(level: 'Hashable | Sequence[Hashable] | None' = None, drop: 'bool' = False, inplace: 'bool' = False, col_level: 'Hashable' = 0, col_fill: 'Hashable' = '') -> 'DataFrame | None' method of pandas.core.frame.DataFrame instance
    Reset the index, or a level of it.
    
    Reset the index of the DataFrame, and use the default one instead.
    If the DataFrame has a MultiIndex, this method can remove one or more
    levels.
    
    Parameters
    ----------
    level : int, str, tuple, or list, default None
        Only remove the given levels from the index. Removes all levels by
        default.
    drop : bool, default False
        Do not try to insert index into dataframe columns. This resets
        the index to the default integer index.
    inplace : bool, default False
        Modify the DataFrame in place (do not create a new object).
    col_level : int or str, default 0
        If the columns have multiple l

In [120]:
COMP=[]
for f in Res.composition.values:
    COMP.append('HEXOSE:{} HEXNAc:{} Neu5Ac:{}'.format(f.count('HEXOSE'),f.count('HEXNAc'),f.count('Neu5Ac')))
Res['composition']=COMP
Res

observed   calculated     delta                 composition  \
0   314.084100   314.085186  0.001086  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0   314.084100   314.085186  0.001086  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0   477.145300   477.145834  0.000534  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0   477.145300   477.172049  0.026749  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
1   679.216800   679.217382  0.000582  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
1   679.216800   679.217382  0.000582  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0   697.227400   697.227946  0.000546  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
1   771.264200   771.264726  0.000526  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
2   841.269700   841.270205  0.000505  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
2   841.269700   841.270205  0.000505  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
3   915.306436   915.306985  0.000549  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
3   915.306436   915.306985  0.000549  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
2   933.317000   933.317549  0.000549  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0   949.290940   949.291487  0.000547  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0   975.327570   975.328114  0.000544  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0   975.327570   975.351923  0.024353  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0   991.301500   991.302051  0.000551  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0   991.301500   991.325861  0.024361  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
4  1077.388000  1077.359808 -0.028192  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
4  1077.388000  1077.359808 -0.028192  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
3  1136.396373  1136.396922  0.000549  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
1  1137.380389  1137.380937  0.000548  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
1  1137.380389  1137.404747  0.024358  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
1  1152.370310  1152.370859  0.000549  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
1  1153.354326  1153.354875  0.000549  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
1  1153.354326  1153.378684  0.024358  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0  1175.358500  1175.420122  0.061622  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0  1197.401518  1197.402067  0.000549  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0  1213.375455  1213.376004  0.000549  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
5  1280.438632  1280.439181  0.000549  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
5  1280.438632  1280.439181  0.000549  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
4  1298.449200  1298.449745  0.000545  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
2  1314.423134  1314.423683  0.000549  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0  1545.554784  1545.555332  0.000548  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0  1561.528721  1561.529270  0.000549  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
5  1589.544613  1589.545162  0.000549  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
3  1605.518550  1605.519099  0.000549  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0  1611.526557  1611.527106  0.000549  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0  1627.500495  1627.501043  0.000548  HEXOSE:1 HEXNAc:1 Neu5Ac:1   
0  1643.473884  1643.474981  0.001097  HEXOSE:1 HEXNAc:1 Neu5Ac:1   

            formula fragment_type  
0     Na1N1C11H17O8          B+Na  
0     Na1N1C11H17O8        Y-W+Na  
0    Na1N1C17H28O13        C-W+Na  
0       H29C19N2O12           B-W  
1    Na1N2C25H40O18          B+Na  
1    Na1N2C25H40O18        Y-W+Na  
0    Na1N2C25H42O19          Y+Na  
1    Na1N2C28H48O21          Y+Na  
2    Na1N2C31H50O23          B+Na  
2    Na1N2C31H50O23        Y-W+Na  
3    Na1N2C34H56O25          B+Na  
3    Na1N2C34H56O25        Y-W+Na  
2    Na1N2C34H58O26          Y+Na  
0     N2K1C34H58O26           Y+K  
0    Na1N2C36H60O27        A24+Na  
0    Na1N3C36H62O26          C+Na  
0     N2K1C36H60O27         A24+K  
0     N3K1C36H62O26           C+K  
4    Na1N2C40H66O30          B+Na  
4    Na1N2C40H66O30        Y-W+Na  
3    Na1N3C42H71O31          Y+Na  
1    Na1N2C42H70O32        A24+Na  
1    Na1N3C42H72O31          C+Na  
1     N3K1C42H71O31           Y+K  
1     N2K1C42H70O32         A24+K  
1     N3K1C42H72O31           C+K  
0       H75C44N2O34           A02  
0    Na1N2C44H74O34        A02+Na  
0     N2K1C44H74O34         A02+K  
5    Na1N3C48H79O35          B+Na  
5    Na1N3C48H79O35        Y-W+Na  
4    Na1N3C48H81O36     

In [115]:
a=('HEXOSE', 'HEXOSE', 'HEXOSE', 'HEXNAc', 'HEXNAc', 'HEXNAc')
a.count('HEXOSE'),a.count('HEXNAc'),a.count('Neu5Ac')
'HEXOSE:{} HEXNAc:{} Neu5Ac:{}'.format(a.count('HEXOSE'),a.count('HEXNAc'),a.count('Neu5Ac'))

'HEXOSE:3 HEXNAc:3 Neu5Ac:0'